# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QSgE7fXhSeo9dYt1S3OPd5pueNBkGgIO_CBHk0CQcrE&tc=u3Q14cBggmnMlLqbVxVO1FI1778EUBG3L41Tv9HoVfY&cc=GyDEsX2TILA_Avacm1EqWJmzWXL3GBPSYR8VY9qerKQ

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhlhrh2QX6LFIMyMRDGp8PuG_CC7FuyTp_8D2gM989syk8lvNP-WnA

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 13.1 MB/s 
     |████████████████████████████████| 6.3 MB 52.0 MB/s 
     |████████████████████████████████| 16.7 MB 50.6 MB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [5]:
## ------------------------------------------------------- ## 
## 1. Import shapefile and start/end dates to clip dataset ##
## ------------------------------------------------------- ## 

start_end = pd.read_csv('climate_stats/start_end_dates.csv')
shp_file = 'POUs/POUs_EDIT_060622_Merge.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [18]:
## ------------------------------------------------- ## 
## IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ## 
## ------------------------------------------------- ## 

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
ir_pr = []
mean_max = []
et_irrig = []

# Import image collection, subset to shapefile, take a statistics for a period of time, 
# and append the image to the designated list
for i in range(len(start_end)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters 
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  ir_pr.append(pr)
  mean_max.append(mxtmp) 
  et_irrig.append(et)

# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)

In [21]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

# This will provide insight into antecedent moisture conditions for a POU.

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

In [22]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/JAtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ir_tmp_stats.csv')
emap.zonal_statistics(ir_tmp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(ir_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ant_precip_stats.csv')
emap.zonal_statistics(ant_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/JAtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/ir_tmp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/irrig_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/ant_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et.csv


In [24]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
ir_precip = pd.read_csv('climate_stats/irrig_precip_stats.csv')
ant_precip = pd.read_csv('climate_stats/ant_precip_stats.csv')
JA_temp = pd.read_csv('climate_stats/JAtemp_stats.csv')
irrig_temp = pd.read_csv('climate_stats/ir_tmp_stats.csv')
et_irrig = pd.read_csv('climate_stats/et.csv')


names = et_irrig['WaterRight']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['irrig_prcp'] = ir_precip.iloc[i,0:34].values 
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Andrews,93.149656,84.23206,27.66706,30.86208,0.790514
1,1988,Andrews,104.597045,68.048646,27.473194,32.672246,0.793502
2,1989,Andrews,196.258921,108.219621,26.105711,31.477965,0.620083
3,1990,Andrews,49.204056,136.184194,26.393551,31.65148,0.842326
4,1991,Andrews,66.634154,142.997269,25.836531,31.404859,0.821943
5,1992,Andrews,105.404923,90.97712,27.377155,31.580158,0.892358
6,1993,Andrews,203.832479,159.370379,24.460559,27.220322,0.759522
7,1994,Andrews,92.223213,103.113442,27.318897,32.895048,0.898363
8,1995,Andrews,186.084199,148.906506,25.17153,29.885366,0.822578
9,1996,Andrews,160.5121,127.28829,25.844812,32.171572,0.798023


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Atwell,104.770001,69.54,27.47299,30.643847,0.999468
1,1988,Atwell,103.720001,73.23,27.268972,32.456043,0.833856
2,1989,Atwell,227.44,95.27,26.030188,31.427252,1.083187
3,1990,Atwell,59.81,134.08,26.321409,31.591759,0.978615
4,1991,Atwell,77.8,152.839999,25.703178,31.283627,1.032139
5,1992,Atwell,109.49,85.490001,27.268692,31.402967,0.843722
6,1993,Atwell,217.800001,147.85,24.340654,27.067034,0.972002
7,1994,Atwell,99.129998,99.71,27.193739,32.727253,1.063766
8,1995,Atwell,201.789999,158.57,25.107803,29.81967,1.053754
9,1996,Atwell,195.619999,130.39,25.73243,32.045494,1.130469


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Ballentyne,133.053746,50.129159,27.101592,30.360606,0.910556
1,1988,Ballentyne,132.215098,86.816434,26.971579,32.28887,0.943185
2,1989,Ballentyne,255.833483,100.259227,25.811107,31.421353,0.934641
3,1990,Ballentyne,90.615503,178.380841,25.864552,31.32197,1.002675
4,1991,Ballentyne,112.069549,131.005608,25.186737,31.040294,0.922014
5,1992,Ballentyne,123.277808,105.85232,26.785658,30.939893,0.963915
6,1993,Ballentyne,246.201141,174.435901,23.753562,26.50369,0.815402
7,1994,Ballentyne,116.096659,108.444549,26.66681,32.416002,0.95347
8,1995,Ballentyne,244.672193,161.094347,24.558363,29.514785,0.952583
9,1996,Ballentyne,284.738897,136.934722,25.144409,31.738827,0.891035


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Barber,131.842963,52.507408,27.079862,30.356307,0.950657
1,1988,Barber,135.036112,87.183705,26.976059,32.299286,0.853515
2,1989,Barber,256.299445,101.682777,25.810043,31.426821,0.901753
3,1990,Barber,90.314814,178.904259,25.846784,31.313177,0.99315
4,1991,Barber,114.972407,132.673704,25.174786,31.049939,0.974864
5,1992,Barber,123.473889,105.869445,26.818577,30.973395,0.958302
6,1993,Barber,241.342407,173.792594,23.750377,26.512885,0.871844
7,1994,Barber,113.077593,106.845557,26.669952,32.423955,1.158989
8,1995,Barber,242.586665,161.063333,24.537027,29.491823,1.094653
9,1996,Barber,286.741481,132.13926,25.112408,31.698609,1.074976


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Bates,92.609288,84.800088,27.672476,30.864997,0.821221
1,1988,Bates,104.395422,67.998933,27.481932,32.678832,0.828963
2,1989,Bates,194.493377,106.953689,26.106027,31.475972,0.616532
3,1990,Bates,48.006044,135.185378,26.394073,31.652389,0.737876
4,1991,Bates,66.300489,144.211112,25.837454,31.405197,0.838548
5,1992,Bates,103.881911,90.217467,27.383037,31.585589,0.727363
6,1993,Bates,201.984577,158.939111,24.462163,27.225711,0.708887
7,1994,Bates,91.062977,102.905244,27.329796,32.907589,0.806937
8,1995,Bates,183.734889,148.447867,25.175032,29.889731,0.836398
9,1996,Bates,157.825555,124.577911,25.853084,32.169616,0.881964


...

,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Thurman Mill,137.113928,52.562109,27.127175,30.42842,0.913256
1,1988,Thurman Mill,147.455776,89.41266,27.013993,32.371372,0.950502
2,1989,Thurman Mill,276.093154,103.46033,25.81671,31.495637,0.955299
3,1990,Thurman Mill,101.991879,179.045613,25.814855,31.330642,0.957797
4,1991,Thurman Mill,119.365791,142.21004,25.123104,31.085228,0.886774
5,1992,Thurman Mill,126.005089,112.732668,26.803304,30.969245,0.916836
6,1993,Thurman Mill,251.037348,175.706722,23.692918,26.489669,0.765877
7,1994,Thurman Mill,118.560107,101.362665,26.673111,32.489024,0.891938
8,1995,Thurman Mill,252.337698,164.131968,24.453635,29.469209,0.918101
9,1996,Thurman Mill,300.659369,132.734666,25.032086,31.686617,0.891389


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Upper Centerpoint,100.681619,72.833479,27.534705,30.704086,0.72867
1,1988,Upper Centerpoint,102.159202,71.584311,27.332573,32.515047,0.817235
2,1989,Upper Centerpoint,220.862549,94.90058,26.064173,31.447489,0.652329
3,1990,Upper Centerpoint,57.645875,130.125252,26.364976,31.627582,0.844552
4,1991,Upper Centerpoint,74.79198,153.251444,25.744456,31.307861,0.796287
5,1992,Upper Centerpoint,106.975678,84.583567,27.318352,31.459345,0.826871
6,1993,Upper Centerpoint,211.24162,149.201652,24.391231,27.124166,0.83577
7,1994,Upper Centerpoint,96.527943,99.323239,27.24817,32.777292,0.927432
8,1995,Upper Centerpoint,197.484541,157.187834,25.154492,29.857409,0.846427
9,1996,Upper Centerpoint,188.050635,128.082134,25.78681,32.088011,0.850904


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Vale Pump,103.551157,70.47033,27.500709,30.669491,0.695974
1,1988,Vale Pump,103.113801,72.359422,27.298624,32.482774,0.606382
2,1989,Vale Pump,225.368264,94.756034,26.04635,31.436079,0.638936
3,1990,Vale Pump,59.098099,132.885537,26.345131,31.611776,0.824059
4,1991,Vale Pump,76.746611,152.845124,25.723214,31.294055,0.742342
5,1992,Vale Pump,108.430578,84.69752,27.295763,31.431132,0.681719
6,1993,Vale Pump,214.708761,147.944298,24.367992,27.096527,0.832078
7,1994,Vale Pump,98.450661,99.446859,27.221146,32.749513,0.944744
8,1995,Vale Pump,201.130083,158.173883,25.134909,29.840802,0.932463
9,1996,Vale Pump,192.721983,129.466281,25.761055,32.067206,0.967316


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Wagner,105.52,64.63,27.428177,30.594397,0.745422
1,1988,Wagner,103.51,71.56,27.238224,32.422855,0.69639
2,1989,Wagner,203.78,92.96,26.028412,31.434505,0.705216
3,1990,Wagner,59.72,142.9,26.30674,31.586264,0.73848
4,1991,Wagner,86.12,152.120001,25.676868,31.269781,0.587105
5,1992,Wagner,109.269999,88.97,27.263317,31.381428,0.643071
6,1993,Wagner,216.98,158.81,24.324018,27.047142,0.619052
7,1994,Wagner,102.67,109.550002,27.174673,32.701099,0.866615
8,1995,Wagner,211.58,152.290001,25.100561,29.815495,0.73385
9,1996,Wagner,208.020001,131.600001,25.709299,32.028572,0.723061


,Year,DIV_NAME,ant_prcp,irrig_prcp,irrig_temp,JuneAug_temp,et
0,1987,Warm Springs,135.242931,51.733576,27.112457,30.39189,1.074633
1,1988,Warm Springs,137.154123,87.470726,27.010789,32.340458,1.063751
2,1989,Warm Springs,260.719438,99.942041,25.818852,31.445234,1.084174
3,1990,Warm Springs,93.109945,181.522424,25.858999,31.334732,1.061571
4,1991,Warm Springs,114.000986,135.529932,25.182217,31.07256,1.073095
5,1992,Warm Springs,124.282781,106.495137,26.814608,30.968933,1.102453
6,1993,Warm Springs,243.680836,174.554234,23.756706,26.524097,0.786771
7,1994,Warm Springs,115.030959,104.239507,26.689377,32.455408,1.024134
8,1995,Warm Springs,245.324958,160.774274,24.545226,29.514194,1.082293
9,1996,Warm Springs,289.204521,131.821384,25.118169,31.724796,1.090542
